# Benchmarking Quasi-Probabilistic Readout Correction of Mid-Circuit Measurements with Randomized Benchmarking (RB) Circuits

In [1]:
import cirq
import numpy as np

1. Construct error-corrected RB circuits from Cirq functions

In [30]:
from cirq import InsertStrategy
from cirq.experiments.qubit_characterizations import (
    _find_inv_matrix,
    _single_qubit_cliffords,
    _two_qubit_clifford,
    _two_qubit_clifford_matrices,
)

We'll use 2-qubit RB circuits and 3-qubit repetition code. Use a single, shallow circuit for testing and visualization purposes. 

In [3]:
num_logical = 2
physical_qubits = 3 * num_logical # repetition code
ancillas = 2 * num_logical
n_qubits = physical_qubits + ancillas
seed = 1
num_cliffords = 3
qubits = cirq.LineQubit.range(n_qubits)
cliffords = _single_qubit_cliffords()
rng = np.random.RandomState(seed)
trials = 1

Set up encoding with 3-qubit repetition code

In [23]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9 = list(qubits)
encoded_subcircuit0_tree= cirq.CNOT(q0, q1), cirq.CNOT(q0, q2)
encoded_subcircuit1_tree = cirq.CNOT(q5, q6), cirq.CNOT(q5, q7)
encoded_subcircuit = cirq.Circuit([encoded_subcircuit0_tree, encoded_subcircuit1_tree])

Construct detection subcircuit to be inserted into RB circuit

In [21]:
detect_subcircuit0_tree = [cirq.CNOT(q0, q4), cirq.CNOT(q1, q4), cirq.CNOT(q1, q3), cirq.CNOT(q2, q3), cirq.CNOT(q3, q4), cirq.measure([q3, q4])]
detect_subcircuit1_tree = [cirq.CNOT(q5, q9), cirq.CNOT(q6, q9), cirq.CNOT(q6, q8), cirq.CNOT(q7, q8), cirq.CNOT(q8, q9), cirq.measure([q8, q9])]
detect_subcircuit = cirq.Circuit([detect_subcircuit0_tree, detect_subcircuit1_tree])

Specify the circuit with random Cliffords, append the detection operations, append the inverse circuit operations, and append a second round of detection. 

In [32]:
clifford_group_size = 11520
log_qubits = [qubits[0], qubits[int((physical_qubits + ancillas)/num_logical)]]
cfd_matrices = _two_qubit_clifford_matrices(
            log_qubits[0],
            log_qubits[1],
            cliffords,
        )
rb_circuits = []
for _ in range(trials):
    idx_list = list(rng.choice(clifford_group_size, num_cliffords))
    rb_circuit = cirq.Circuit()
    for idx in idx_list:
        rb_circuit.append(
            _two_qubit_clifford(log_qubits[0], log_qubits[1], idx, cliffords)
        )
    inv_idx = _find_inv_matrix(
        cirq.protocols.unitary(rb_circuit), cfd_matrices
    )
    rb_circuit.append(detect_subcircuit)
    rb_circuit.append(
        _two_qubit_clifford(log_qubits[0], log_qubits[1], inv_idx, cliffords), strategy=InsertStrategy.INLINE,
    )
    rb_circuit.append(detect_subcircuit)
    rb_circuits.append(rb_circuit)

Combine the sub-circuits and print the circuit.

In [33]:
corrected_circ = cirq.Circuit([encoded_subcircuit, rb_circuit])
corrected_circ

0: ───@───@───Y^-0.5───X^0.5────Y^0.5───@───Y^0.5────@───Y────────X^0.5───Y───────X^0.5───@───X^-0.5───Y^-0.5───X─────Y^0.5───@───X^-0.5───Y^-0.5───@───────────────────Y^-0.5───X^0.5────────────@───Y^0.5────@───Y^0.5───@───────────────────────
      │   │                             │            │                                    │                                   │                     │                                             │            │           │
1: ───X───┼─────────────────────────────┼────────────┼────────────────────────────────────┼───────────────────────────────────┼─────────────────────┼───@───@─────────────────────────────────────┼────────────┼───────────┼───@───@───────────────
          │                             │            │                                    │                                   │                     │   │   │                                     │            │           │   │   │
2: ───────X─────────────────────────────┼────────────┼────────────────────────────────────┼───────────────────────────────────┼─────────────────────┼───┼───┼───@─────────────────────────────────┼────────────┼───────────┼───┼───┼───@───────────
                                        │            │                                    │                                   │                     │   │   │   │                                 │            │           │   │   │   │
3: ─────────────────────────────────────┼────────────┼────────────────────────────────────┼───────────────────────────────────┼─────────────────────┼───┼───X───X───@───M─────────────────────────┼────────────┼───────────┼───┼───X───X───@───M───
                                        │            │                                    │                                   │                     │   │           │   │                         │            │           │   │           │   │
4: ─────────────────────────────────────┼────────────┼────────────────────────────────────┼───────────────────────────────────┼─────────────────────X───X───────────X───M─────────────────────────┼────────────┼───────────X───X───────────X───M───
                                        │            │                                    │                                   │                                                                   │            │
5: ───@───@───Y────────X^-0.5───────────@───X^-0.5───@───Y^-0.5───Y^0.5───X^0.5───Y^0.5───@───Y^0.5────Y^0.5────X^0───────────@───Y────────X^0.5────@────────────────────────────Y^0.5───X^-0.5───@───X^-0.5───@───X^0.5───@───────────────────────
      │   │                                                                                                                                         │                                                                      │
6: ───X───┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───@───@──────────────────────────────────────────────────────────────┼───@───@───────────────
          │                                                                                                                                         │   │   │                                                              │   │   │
7: ───────X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───┼───┼───@──────────────────────────────────────────────────────────┼───┼───┼───@───────────
                                                                                                                                                    │   │   │   │                                                          │   │   │   │
8: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───┼───X───X───@───M──────────────────────────────────────────────────┼───┼───X───X

2. Add coherent measurement errors

3. Apply Randomized Compiling before measurement

4. Apply PEC after RC and before measurement

5. Once this workflow is working, try noise scaling at, e.g. different sampling overheads, noise levels in representations, etc.